In [28]:
import tensorflow as tf 
import os
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import imageio as iio
import cv2

# Keras Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [2]:
data = tf.keras.utils.image_dataset_from_directory("data")

Found 5631 files belonging to 4 classes.


In [3]:
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [4]:
# Number represents numpy array of images and the labels
scaled = batch[0]/255 

In [5]:
scaled.max()

1.0

# 2. PreProcess Data

In [6]:
data = data.map(lambda x, y: (x/255, y))

In [7]:
scaled_iterator = data.as_numpy_iterator()

In [8]:
scaled_iterator.next()[0].max()

0.94509804

### 2.1 Split Data

In [9]:
len(data)

176

In [10]:
train_size = int(len(data) * .7)
val_size = int(len(data) *.2) + 1
test_size = int(len(data) *.1) + 1

In [11]:
train = data.take(train_size)
test = data.take(test_size)
val = data.take(val_size)

# 3. Deep Model

## 3.1 Build Deep Learning Model

In [12]:
model = Sequential()

In [13]:
model.add(Conv2D(16, (3, 3), 1, activation = "relu", input_shape = (256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3), 1, activation = "relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3), 1, activation = "relu"))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation = "relu"))
model.add(Dense(4, activation = "softmax"))

In [18]:
model.compile("adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 64)        0

## 3.2 Train

In [20]:
logdir = "logs"
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

In [21]:
hist = model.fit(train, epochs = 20, validation_data=val, callbacks = [tensorboard_callback])

Epoch 1/20




123/123 [==============================] - 45s 359ms/step - loss: 0.4066 - accuracy: 0.8194 - val_loss: 0.1701 - val_accuracy: 0.9358
Epoch 2/20
123/123 [==============================] - 43s 347ms/step - loss: 0.2216 - accuracy: 0.8989 - val_loss: 0.1773 - val_accuracy: 0.9253
Epoch 3/20
123/123 [==============================] - 46s 373ms/step - loss: 0.1788 - accuracy: 0.9207 - val_loss: 0.1861 - val_accuracy: 0.9253
Epoch 4/20
123/123 [==============================] - 52s 422ms/step - loss: 0.1931 - accuracy: 0.9090 - val_loss: 0.1464 - val_accuracy: 0.9358
Epoch 5/20
123/123 [==============================] - 48s 388ms/step - loss: 0.1687 - accuracy: 0.9167 - val_loss: 0.2538 - val_accuracy: 0.8958
Epoch 6/20
123/123 [==============================] - 50s 405ms/step - loss: 0.1706 - accuracy: 0.9271 - val_loss: 0.2062 - val_accuracy: 0.9028
Epoch 7/20
123/123 [==============================] - 48s 387ms/step - loss: 0.1631 - accuracy: 0.9266 - val_loss: 0.1397 - val_accuracy: 0

In [22]:
# Images Prediction

In [81]:
def predict_photo(image_dir):
    img = iio.imread(image_dir)
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
    prediction = model.predict(np.array([img / 255]))
    label_map = {0: "Cloudy", 1: "Desert", 2:"Green Area", 3:"Water"}
    label = prediction.argmax()
    print(label_map.get(label))
    

In [84]:
predict_photo("C:/traffic_sign_image_classification/satellite_desert.jpg")

1/1 [==============================] - 0s 36ms/step
desert


C:\Users\Jevan\AppData\Local\Temp\ipykernel_23036\1166731189.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread(image_dir)
